Note: this notebook has been modified specifically for use on the oxford data set!

In [1]:
import csv
import pandas as pd
import numpy as np
import random
from hodgerank_tools import *

## 1. Preprocessing
### Create data

In [2]:
k_sev=1376
k_non_sev=4324
sev_p=[0.884,0.711,0.603,0.442,0.376,0.357,0.26,0.26,0.161,0.113,0.078,
       0.059,0.057,0.028]
non_sev_p=[0.814,0.657,0.442,0.057,0.28,0.128,0.131,0.109,0.121,0.135,0.097,
           0.057,0.058,0.051]
sev_n=[round(k_sev*i) for i in sev_p]
non_sev_n=[round(k_non_sev*i) for i in non_sev_p]
naive_r = [(sev_n[i]+non_sev_n[i])/5700 for i in range(len(non_sev_p))]
print(naive_r)
data = [sev_n, non_sev_n]
symptoms = ["Fever", "Cough", "Fatigue", "Dyspnea", "Sputum", "Shortness of Breath", "Myalgia", "Chill", "Dizziness", "Headache", "Sore Throat", "Nausea", "Diarhea", "Congestion"]
covid_df = pd.DataFrame(data, columns = symptoms)

print(covid_df)

[0.8308771929824561, 0.67, 0.4808771929824561, 0.14982456140350878, 0.3031578947368421, 0.1831578947368421, 0.16210526315789472, 0.14543859649122806, 0.1307017543859649, 0.12964912280701754, 0.09228070175438596, 0.057368421052631575, 0.05771929824561404, 0.0456140350877193]
   Fever  Cough  Fatigue  Dyspnea  Sputum  Shortness of Breath  Myalgia  \
0   1216    978      830      608     517                  491      358   
1   3520   2841     1911      246    1211                  553      566   

   Chill  Dizziness  Headache  Sore Throat  Nausea  Diarhea  Congestion  
0    358        222       155          107      81       78          39  
1    471        523       584          419     246      251         221  


In [3]:
# multiply all values by scalar, with sd, around normal dist
def norm(center, sd):
    rating = np.abs(np.random.normal(center, sd)) 
    if rating > 10: rating = 10
    return rating

In [4]:
big_sev_list = []
big_non_sev_list = []

for symptom in symptoms:
    ones = [norm(8, 2) for i in range(covid_df[symptom][0])]
    zeroes = [0 for i in range(k_sev - covid_df[symptom][0])]
    new_col = ones + zeroes
    random.shuffle(new_col)
    big_sev_list.append(new_col)
    
for symptom in symptoms:
    ones = [norm(3,1) for i in range(covid_df[symptom][1])]
    zeroes = [0 for i in range(k_non_sev - covid_df[symptom][1])]
    new_col = ones + zeroes
    random.shuffle(new_col)
    big_non_sev_list.append(new_col)
    
big_sev_array = np.array(big_sev_list)
big_non_sev_array = np.array(big_non_sev_list)

big_sev_array = np.transpose(big_sev_array)
big_non_sev_array = np.transpose(big_non_sev_array)

print(big_sev_array.shape)
print(big_non_sev_array.shape)
big_data = np.concatenate((big_sev_array, big_non_sev_array), axis=0)
print(big_data.shape)
covid_df = pd.DataFrame(big_data, columns = symptoms)
print(covid_df.head)

#save final version
#covid_df.to_csv('data/covid/covid_df.csv')

(1376, 14)
(4324, 14)
(5700, 14)
<bound method NDFrame.head of          Fever      Cough   Fatigue    Dyspnea    Sputum  Shortness of Breath  \
0     0.000000   7.566467  8.326692  10.000000  0.000000             0.000000   
1     0.000000  10.000000  8.032917   0.000000  0.000000            10.000000   
2     7.474325   8.375201  0.000000   8.396149  0.000000             0.000000   
3     5.908051   7.615881  0.000000   8.017046  8.176538             0.000000   
4     9.904310   0.000000  0.000000   0.000000  0.000000             9.524028   
...        ...        ...       ...        ...       ...                  ...   
5695  3.028602   4.722897  0.000000   0.000000  1.912452             0.000000   
5696  4.212678   0.000000  0.000000   0.000000  3.665124             0.000000   
5697  0.000000   0.000000  0.000000   0.000000  0.000000             0.000000   
5698  1.726451   3.402318  3.888512   0.000000  0.000000             0.000000   
5699  0.966311   0.000000  0.000000   0.000000

In [9]:
covid_df = pd.read_csv("data/covid/covid_df.csv")[symptoms]
print(covid_df.head())

      Fever      Cough   Fatigue    Dyspnea    Sputum  Shortness of Breath  \
0  0.000000   7.566467  8.326692  10.000000  0.000000             0.000000   
1  0.000000  10.000000  8.032917   0.000000  0.000000            10.000000   
2  7.474325   8.375201  0.000000   8.396149  0.000000             0.000000   
3  5.908051   7.615881  0.000000   8.017046  8.176538             0.000000   
4  9.904310   0.000000  0.000000   0.000000  0.000000             9.524028   

    Myalgia     Chill  Dizziness  Headache  Sore Throat  Nausea   Diarhea  \
0  0.000000  0.000000   0.000000       0.0     0.000000     0.0  0.000000   
1  0.000000  0.000000   5.946329       0.0     0.000000     0.0  0.000000   
2  0.000000  8.545903   0.000000       0.0     9.144488     0.0  6.870031   
3  4.334554  0.000000   0.000000       0.0     0.000000     0.0  0.000000   
4  6.945133  6.546888   7.962847       0.0     0.000000     0.0  0.000000   

   Congestion  
0    0.000000  
1    0.000000  
2    0.000000  
3   

## 2. Graph Building
14 nodes

In [10]:
(covid_rank, covid_rank_error) = rank(covid_df)
print(covid_rank)
print("error: ", covid_rank_error)

                element         r
0                 Fever  2.414563
1                 Cough  1.729239
2               Fatigue  1.047327
3                Sputum  0.247917
4   Shortness of Breath -0.129866
5               Dyspnea -0.137169
6               Myalgia -0.318367
7                 Chill -0.361105
8             Dizziness -0.517530
9              Headache -0.580460
10          Sore Throat -0.731391
11              Diarhea -0.864645
12               Nausea -0.867256
13           Congestion -0.931256
error:  1.0595062756095328e-27


In [11]:
#save final version
covid_rank.to_csv('data/covid/covid_rank.csv')